In [2]:
import pandas as pd
data = pd.read_csv('time_cluster.csv')

In [1]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg



In [49]:
import numpy as np
import pandas as pd
import pickle
md = pickle.load(open('K-meas.pkl', 'rb'))

index = pd.Series(np.arange(0, 1996))
r = []
for i in range(0, 7):
    ret = []
    for j in index[(md.labels_ == i)]:
        data_ = pd.read_csv(f'No_abnormality_time/time{j}.csv')
        n_in = 30
        n_out = 0
        # 将时序数据转换为监督问题数据
        reframed = series_to_supervised(data_.qty[-31:].values.reshape(-1, 1), n_in, n_out)    
        reframed['product_no'] = data_['product_no']
        reframed['seller_no'] = data_['seller_no']
        reframed['warehouse_no'] = data_['warehouse_no']
        ret.append(reframed)
    new = pd.concat(ret)
    new['label'] = i
    r.append(new)
pickle.dump(pd.get_dummies(pd.concat(r), columns=['label']), open(f'timeseries_test.pkl', 'wb'))

In [50]:
pickle.load(open('timeseries_test.pkl', 'rb'))

,var1(t-30),var1(t-29),var1(t-28),var1(t-27),var1(t-26),var1(t-25),var1(t-24),var1(t-23),var1(t-22),var1(t-21),...,product_no,seller_no,warehouse_no,label_0,label_1,label_2,label_3,label_4,label_5,label_6
30,1.0,4.0,3.0,1.0,3.0,4.0,2.0,4.0,2.0,1.0,...,product_501,seller_21,wh_1,True,False,False,False,False,False,False
30,3.0,3.0,4.0,1.0,7.0,1.0,7.0,3.0,4.0,3.0,...,product_479,seller_21,wh_1,True,False,False,False,False,False,False
30,4.0,2.0,6.0,1.0,4.0,3.0,11.0,7.0,1.0,6.0,...,product_494,seller_21,wh_1,True,False,False,False,False,False,False
30,12.0,10.0,12.0,29.0,21.0,18.0,18.0,11.0,16.0,17.0,...,product_489,seller_21,wh_1,True,False,False,False,False,False,False
30,0.0,2.0,1.0,0.0,2.0,1.0,4.0,3.0,2.0,0.0,...,product_1654,seller_21,wh_1,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,2.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,product_431,seller_19,wh_21,False,False,False,False,False,False,True
30,4.0,2.0,3.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,...,product_431,seller_19,wh_28,False,False,False,False,False,False,True
30,2.0,1.0,3.0,1.0,5.0,1.0,2.0,3.0,4.0,0.0,...,product_438,seller_19,wh_28,False,False,False,False,False,False,True
30,4.0,3.0,4.0,3.0,8.0,3.0,4.0,10.0,2.0,1.0,...,product_438,seller_19,wh_11,False,False,False,False,False,False,True


In [3]:
import numpy as np
import pandas as pd
import pickle
labels = pd.read_csv('time_cluster.csv')['labels']

index = pd.Series(np.arange(0, 1996))
r = []
for i in range(0, 7):
    ret = []
    for j in index[(labels == i)]:
        data_ = pd.read_csv(f'No_abnormality_time/time{j}.csv')
        n_in = 7
        n_out = 4
        # 将时序数据转换为监督问题数据
        reframed = series_to_supervised(data_.qty.values.reshape(-1, 1), n_in, n_out)    
        
        ret.append(reframed)
    new = pd.concat(ret)
    new['label'] = i
    r.append(new)
_ = pd.get_dummies(pd.concat(r), columns=['label'])
_['promotion'] = False
pickle.dump(_, open(f'timeseries_10.pkl', 'wb'))